In [3]:
from keras.layers import Activation, Conv2D, Dense, Flatten, MaxPool2D, Dropout
from keras.models import Sequential, load_model
from keras.utils.np_utils import to_categorical
from keras.utils.vis_utils import plot_model
import keras
from keras.datasets import mnist
from sklearn.model_selection import train_test_split

from core.preprocessing import read_data, spectra_image
from utils.labeling import put_labels
from utils.bmp_csv import bmpToCSV
from utils.make_train_data import parsed_data

from tensorflow.keras import optimizers
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
label_file = "../data/ans_type0.csv"
x_dir_path = "../data/atom_linear_spectrum/"

x_data, y_data = parsed_data(label_file, x_dir_path)
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=2)

print(X_train.shape)

100%|██████████| 100/100 [00:00<00:00, 7711.39it/s]

(80, 255, 640)


In [9]:
X_train = X_train.reshape(-1, 640, 255, 1)
X_test = X_test.reshape(-1, 640, 255, 1)
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), input_shape=(640,255,1)))
model.add(Activation("relu"))
model.add(Conv2D(filters=64, kernel_size=(3,3)))
model.add(Conv2D(filters=64, kernel_size=(3,3)))
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(640))

model.compile(loss="logloss", optimizer="adadelta", metrics=["accuracy"])
model.fit(X_train, y_train, batch_size = 2, epochs=10, verbose = -1, validation_data = (X_test, y_test))

scores = model.evaluate(X_test, y_test, verbose=1)
print("Test loss: ", scores[0])
print("Test accuracy: ", scores[1])

ResourceExhaustedError: OOM when allocating tensor with shape[25786048,640] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:RandomUniform]